# Structure Representations

The [DiffPy-CMI](http://www.diffpy.org/products/diffpycmi/) framework provides several options for representing atomic structures of materials.  Here we demonstrate basic operations on structure models using the [diffpy.structure](https://github.com/diffpy/diffpy.structure) and [pyobjcryst](https://github.com/diffpy/pyobjcryst) Python packages included in the DiffPy-CMI software.

**Contents**

> [diffpy.structure](#diffpy.structure)<br>
> [pyobjcryst](#pyobjcryst)

## diffpy.structure

In [diffpy.structure](https://github.com/diffpy.structure) the atom arrangements are represented as a collaboration of *Structure*, *Atom* and *Lattice* classes.  The *Structure* class is an enhanced Python list of *Atom* objects, where each atom stores fractional coordinates, element or ion symbol, a matrix of displacement parameters and other attributes.  The *Lattice* class defines fractional coordinates with respect to the absolute Cartesian coordinate system and provides functions for conversion between fractional and Cartesian coordinates and other geometric operations.

In [1]:
from __future__ import print_function
from diffpy.structure import Structure, Atom, Lattice

acs = Atom('Cs', [0, 0, 0])
acl = Atom('Cl', [0.5, 0.5, 0.5])
cscl = Structure(atoms=[acs, acl],
                 lattice=Lattice(4.123, 4.123, 4.123, 90, 90, 90))

print(cscl)

lattice=Lattice(a=4.123, b=4.123, c=4.123, alpha=90, beta=90, gamma=90)
Cs   0.000000 0.000000 0.000000 1.0000
Cl   0.500000 0.500000 0.500000 1.0000


The atoms in the `cscl` object can be accessed by integer indices starting from zero.  A subset of atoms can be selected using an index range, but also using a list of indices or a mask array of boolean flags.

In [2]:
print(cscl[0], '\n---')     # first atom in the structure
print(cscl[::-1], '\n---')  # reversed order of atoms
print(cscl[[1,]], '\n---')  # substructure containing only the 2nd atom
print(cscl[cscl.x < 0.2])   # substructure of atoms with x < 0.2

Cs   0.000000 0.000000 0.000000 1.0000 
---
lattice=Lattice(a=4.123, b=4.123, c=4.123, alpha=90, beta=90, gamma=90)
Cl   0.500000 0.500000 0.500000 1.0000
Cs   0.000000 0.000000 0.000000 1.0000 
---
lattice=Lattice(a=4.123, b=4.123, c=4.123, alpha=90, beta=90, gamma=90)
Cl   0.500000 0.500000 0.500000 1.0000 
---
lattice=Lattice(a=4.123, b=4.123, c=4.123, alpha=90, beta=90, gamma=90)
Cs   0.000000 0.000000 0.000000 1.0000


Atom positions in the structure are specified in fractional coordinates; therefore, their Cartesian positions and relative distances change with a change in lattice parameters.  Here we show this behavior using the *distance* function and the integer indices of the relevant atoms.

In [3]:
print("Cl Cartesian coordinates:", cscl[1].xyz_cartn)
print("Cs-Cl distance:", cscl.distance(0, 1))
print("# change cell parameter to a = 3")
cscl.lattice.setLatPar(a=3, b=3, c=3)
print("Cl Cartesian coordinates:", cscl[1].xyz_cartn)
print("Cs-Cl distance:", cscl.distance(0, 1))

Cl Cartesian coordinates: [2.0615 2.0615 2.0615]
Cs-Cl distance: 3.570622739803241
# change cell parameter to a = 3
Cl Cartesian coordinates: [1.5 1.5 1.5]
Cs-Cl distance: 2.598076211353316


Details about each site are stored as data attributes of the *Atom* object.  These attributes can be changed either individually per each atom or using mapped arrays of the owning *Structure*.  Thus, rather than using a for loop, a single statement can be used to set isotropic displacement parameters for all atoms or for a subset of chlorine atoms.

In [4]:
cscl.Uisoequiv = 0.003
print(cscl.Uisoequiv)
cscl[cscl.element == 'Cl'].Uisoequiv = 0.004
print(cscl.Uisoequiv) 

[0.003 0.003]
[0.003 0.004]


The changes in the equivalent isotropic displacement parameter *Uisoequiv* are propagated to the tensor of displacement parameters *U*.  The *anisotropy* flag specifies whether anisotropic displacements are allowed on each atom site.

In [5]:
print(cscl.U, '\n---')
print(cscl.anisotropy)

[[[0.003 0.    0.   ]
  [0.    0.003 0.   ]
  [0.    0.    0.003]]

 [[0.004 0.    0.   ]
  [0.    0.004 0.   ]
  [0.    0.    0.004]]] 
---
[False False]


In [6]:
# allow anisotropy for the first atom
cscl[0].anisotropy = True
cscl[0].U11 = 0.004
print(cscl.U, '\n---')
print(cscl.anisotropy)

[[[0.004 0.    0.   ]
  [0.    0.003 0.   ]
  [0.    0.    0.003]]

 [[0.004 0.    0.   ]
  [0.    0.004 0.   ]
  [0.    0.    0.004]]] 
---
[ True False]


Structure data can be loaded from several file formats such as XYZ, PDB or CIF.  The *Structure* class has been designed as a simple list of atoms with no awareness of crystal symmetry.  Therefore, when loading from a CIF file, the asymmetric unit gets expanded to a full unit cell as if in the P1 symmetry.

In [7]:
from diffpy.structure import loadStructure
nacl = loadStructure('NaCl.cif')
print(nacl)

lattice=Lattice(a=5.62, b=5.62, c=5.62, alpha=90, beta=90, gamma=90)
Na1+ 0.000000 0.000000 0.000000 1.0000
Na1+ 0.000000 0.500000 0.500000 1.0000
Na1+ 0.500000 0.000000 0.500000 1.0000
Na1+ 0.500000 0.500000 0.000000 1.0000
Cl1- 0.500000 0.500000 0.500000 1.0000
Cl1- 0.500000 0.000000 0.000000 1.0000
Cl1- 0.000000 0.500000 0.000000 1.0000
Cl1- 0.000000 0.000000 0.500000 1.0000


In [8]:
print(nacl.label)

['Na1' 'Na1_2' 'Na1_3' 'Na1_4' 'Cl1' 'Cl1_2' 'Cl1_3' 'Cl1_4']


Although symmetry operations are not intrinsic to the *Structure* class, the diffpy.structure package provides functions for generating symmetry equivalent positions or symmetry constraints for the coordinates and displacement parameters.  The package also provides definitions for all space groups in over 500 different symmetry settings.

In [9]:
from diffpy.structure.spacegroups import GetSpaceGroup
from diffpy.structure.symmetryutilities import ExpandAsymmetricUnit

Fm3m = GetSpaceGroup('Fm-3m')
eau = ExpandAsymmetricUnit(Fm3m, corepos=[[0, 0, 0],])
eau.expandedpos

[[array([0., 0., 0.]),
  array([0. , 0.5, 0.5]),
  array([0.5, 0. , 0.5]),
  array([0.5, 0.5, 0. ])]]

The *SymmetryConstraints* class can be used to determine symmetry constraints on positions and displacement tensors at the specified unit cell sites.

In [10]:
from diffpy.structure.symmetryutilities import SymmetryConstraints

symcon = SymmetryConstraints(Fm3m, positions=nacl.xyz)

print('asymmetric unit and the equivalent positions:', symcon.coremap)
print('position parameters:', symcon.pospars)
print('position constraints on the 1-st site:', symcon.poseqns[0])
print('ADP parameters:', symcon.Upars)
print('ADP constraints on the 1-st site:', symcon.Ueqns[0])

asymmetric unit and the equivalent positions: {0: [0, 1, 2, 3], 4: [4, 5, 6, 7]}
position parameters: []
position constraints on the 1-st site: {'x': '+0', 'y': '+0', 'z': '+0'}
ADP parameters: [('U110', 0.0), ('U114', 0.0)]
ADP constraints on the 1-st site: {'U11': 'U110', 'U22': 'U110', 'U33': 'U110', 'U12': '0', 'U13': '0', 'U23': '0'}


For more information about the diffpy.structure package, see the online documentation at http://www.diffpy.org/diffpy.structure.

## pyobjcryst

Another option for describing material structures with DiffPy-CMI is to use the pyobjcryst package.  [pyobjcryst](https://github.com/diffpy/pyobjcryst) is a Python interface to the [ObjCryst++](https://sourceforge.net/projects/objcryst/) crystallographic library developed by Vincent Favre-Nicolin.  pyobjcryst provides advanced features for representing crystal structures with intrinsic crystal symmetries, chemical constraints such as rigid atom groups, and restraints on bond lengths and bond and/or torsion angles.  The pyobjcryst *Crystal* class represents periodic structures as a collection of general scatterers located at fractional coordinates.  Each scatterer is a single atom site in simple case; however, it can also be a collection of atom positions grouped in a *Molecule* object or described by Z-Matrix.  The pyobjcryst package supports the standard CIF and a native XML format for loading and storing structure models. 

In [11]:
from pyobjcryst import loadCrystal

nacl = loadCrystal('NaCl.cif')
print(nacl)

UnitCell : Sodium chloride(F m -3 m)
    Cell dimensions :  5.62000   5.62000   5.62000  90.00000  90.00000  90.00000
List of scattering components (atoms): 2
Na1              at :  0.0000 0.0000 0.0000, Occup=1.0000 * 0.0208, ScattPow:Na1             , Biso=    0.0000
Cl1              at :  0.5000 0.5000 0.5000, Occup=1.0000 * 0.0208, ScattPow:Cl1             , Biso=    0.0000

Occupancy = occ * dyn, where:
        - occ is the 'real' occupancy
        - dyn is the dynamical occupancy correction, indicating  either
          an atom on a special position, or several identical atoms 
          overlapping (dyn=0.5 -> atom on a symetry plane / 2fold axis..
                               -> OR 2 atoms strictly overlapping)

 Total number of components (atoms) in one unit cell : 8


The *CreateCrystalFromCIF* function returned an instance of the pyobjcryst *Crystal* class, which has a variety of functions for accessing and manipulating structure data.  For example, the *GetSpaceGroup()* function provides access to the active space group.

In [12]:
print(nacl.GetSpaceGroup())

SpaceGroup:
  Schoenflies symbol = Oh^5
  Hermann-Maugin symbol = F m -3 m
  Hall symbol = -F 4 2 3
  SgNumber = 225
  Number of Seitz Matrix = 24
  Number of Translation Vectors = 4
  List of Seitz Matrices : 
    x,y,z
    -y,x,z
    -x,-y,z
    y,-x,z
    x,-y,-z
    y,x,-z
    -x,y,-z
    -y,-x,-z
    z,x,y
    -x,z,y
    -z,-x,y
    x,-z,y
    z,-x,-y
    x,z,-y
    -z,x,-y
    -x,-z,-y
    y,z,x
    y,-z,-x
    z,y,-x
    -y,z,-x
    -z,-y,-x
    -y,-z,x
    z,-y,x
    -z,y,x
  There is an inversion center at 0 0 0
  List of Translation vectors :
     0,0,0
     0,0.5,0.5
     0.5,0,0.5
     0.5,0.5,0
Extension (origin choice, rhomboedral/hexagonal): 


The positions of the Na and Cl scatterers are specified in fractional coordinates; therefore, the bond distances and angles change with lattice parameters.

In [13]:
from pyobjcryst.crystal import Crystal

print("ORIGINAL:")
for sc in nacl.GetScatteringComponentList():
    print(sc)
print("Na-Cl distance:", nacl.GetMinDistanceTable()[0, 1])
a2 = 7
nacl.a, nacl.b, nacl.c = a2, a2, a2
print("EXPANDED:")
for sc in nacl.GetScatteringComponentList():
    print(sc)
print("Na-Cl distance:", nacl.GetMinDistanceTable()[0, 1])

ORIGINAL:
0 0 0 1 0.0208333 0x7f8cd3936660 Na1
0.5 0.5 0.5 1 0.0208333 0x7f8cd3936c60 Cl1
Na-Cl distance: 2.81
EXPANDED:
0 0 0 1 0.0208333 0x7f8cd3936660 Na1
0.5 0.5 0.5 1 0.0208333 0x7f8cd3936c60 Cl1
Na-Cl distance: 3.5


Sometimes it is preferable to have bond distances and angles independent of lattice parameters - for example when searching for crystal packing of molecules of a known shape.  The pyobjcryst library allows this by using an entire molecule as a scattering unit.

In [14]:
from pyobjcryst.crystal import Crystal
from pyobjcryst.molecule import Molecule
from pyobjcryst.scatteringpower import ScatteringPowerAtom

crst = Crystal(4, 4, 4, 'P1')
# atom scattering properties are defined by the
# ScatteringPower object.  The same ScatteringPower may
# be shared among several atom sites.
spC = ScatteringPowerAtom('Carbon', 'C')
crst.AddScatteringPower(spC)
# molecules in ObjCryst++ are owned by a Crystal object and
# thus need to be created with a reference to their owner.
mol = Molecule(crst, "mol")
# atom positions are specified in Cartesian coordinates
# anchored at the center of mass of the Molecule.
mol.AddAtom(-1, 0, 0, spC, 'C1')
mol.AddAtom(+1, 0, 0, spC, 'C2')
# activate the Molecule as a scattering entity within the Crystal.
crst.AddScatterer(mol)

print("ORIGINAL:")
for sc in crst.GetScatteringComponentList():
    print(sc)
print("C1-C2 distance:", crst.GetMinDistanceTable()[0, 1])

# enlarge lattice parameters
crst.a, crst.b, crst.c = 8, 8, 8
print("EXPANDED:")
for sc in crst.GetScatteringComponentList():
    print(sc)
print("C1-C2 distance:", crst.GetMinDistanceTable()[0, 1])

ORIGINAL:
-0.25 0 0 1 1 0x7f8cd39d1670 Carbon
0.25 0 0 1 1 0x7f8cd39d1670 Carbon
C1-C2 distance: 2.0
EXPANDED:
-0.125 0 0 1 1 0x7f8cd39d1670 Carbon
0.125 0 0 1 1 0x7f8cd39d1670 Carbon
C1-C2 distance: 2.0


The fractional coordinates of the carbon atoms were adjusted to preserve the same interatomic distances within the Molecule.  The position of a *Molecule* object within a *Crystal* is specified in fractional coordinate; changing this coordinate will affect all atoms in the molecule.  Molecules can also be rotated by changing their orientation Quaternion.

In [15]:
from math import pi
from pyobjcryst.molecule import Quaternion

# quaternions for a null rotation and for a 90deg rotation by the z-axis
qnone = Quaternion.RotationQuaternion(0, 0, 0, 1)
qz90 = Quaternion.RotationQuaternion(0.5 * pi, 0, 0, 1)

# (1) move the Molecule mol along the Y-axis
mol.Y = 0.5
mol.Q0, mol.Q1, mol.Q2, mol.Q3 = qnone.Q0, qnone.Q1, qnone.Q2, qnone.Q3
print("MOVED:")
for sc in crst.GetScatteringComponentList():
    print(sc)
    
# (2) rotate the molecule by 90 degrees around the Z-axis
mol.Q0, mol.Q1, mol.Q2, mol.Q3 = qz90.Q0, qz90.Q1, qz90.Q2, qz90.Q3
print("ROTATED:")
for sc in crst.GetScatteringComponentList():
    print(sc)

MOVED:
-0.125 0.5 0 1 1 0x7f8cd39d1670 Carbon
0.125 0.5 0 1 1 0x7f8cd39d1670 Carbon
ROTATED:
-6.25e-09 0.375 0 1 1 0x7f8cd39d1670 Carbon
6.25e-09 0.625 0 1 1 0x7f8cd39d1670 Carbon


---

For more information about the pyobjcryst package see http://www.diffpy.org/pyobjcryst/.<br>
The underlying ObjCryst++ library is documented at http://vincefn.net/ObjCryst/.